### Setup

In [1]:
# imports for analyses
import csv
import math
import random
import numpy as np
import pandas as pd
import seaborn as sns
import pingouin as pg
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr, ttest_rel
from scipy import spatial
from matplotlib import gridspec
from matplotlib.lines import Line2D
from mycolorpy import colorlist as mcp
cset=mcp.gen_color(cmap="inferno",n=8)
from statsmodels.stats.power import TTestIndPower

In [2]:
# obtain human data details
exp='data/pilotdata_opt/'
data_id=1 #data_id indicates folder for functions_overrides.py to use
ll = 16   #list length
ntrials_persub = 12 #number of expeirmental trials
recall_time = 90000 #ms

# only included trials, following exclusion criteria (see Methods):
recs_amts_data = np.loadtxt(exp+'/recs_amts.txt',delimiter=',') #initial recall amount by trial
cue_times_data = np.loadtxt(exp+'/cue_times.txt',delimiter=',') #cue request time into the 90 seconds
rmdr_idxs_data = np.loadtxt(exp+'/rmdr.txt',delimiter=',')      #index of cue in remaining word list
post_amts_data = np.loadtxt(exp+'/post_rmdr_amt.txt',delimiter=',')   #post-cue recall amount by trial

ntrials = len(recs_amts_data) #number of included trials
ntrials_all = len(np.loadtxt(exp+'/pres_all.txt',delimiter=',').tolist()) #total collected trials
nsubjects = int(ntrials_all/ntrials_persub) #number of subjects overall

print("\n\033[1m --- HUMAN DATA DETAILS --- \n\033[0m")
print(" *",nsubjects,"subjects included in analyses")
print(" *",ntrials,"cued trials out of",ntrials_all,"total trials are included")
print(" * Cue time M = %.1f, SD = %.1f seconds"%(np.mean(cue_times_data)/1000,np.std(cue_times_data)/1000))
print(" * Pre-cue recall M = %.2f, SD = %.2f seconds"%(np.mean(recs_amts_data),np.std(recs_amts_data)))
print(" * Totalled recall M = %.2f, SD = %.2f seconds"%(np.mean(recs_amts_data)+np.mean(post_amts_data),
                                                        np.std(recs_amts_data) + np.std(post_amts_data)))


 --- HUMAN DATA DETAILS --- 

 * 66 subjects included in analyses
 * 523 cued trials out of 792 total trials are included
 * Cue time M = 42.8, SD = 18.7 seconds
 * Pre-cue recall M = 7.68, SD = 3.22 seconds
 * Totalled recall M = 8.48, SD = 4.34 seconds


### <p></p>

### Simulate Model

In [3]:
# import CMR code
from probCMR_overrides import CMR2Reminder
from functions_overrides import FunctionsReminder
functions = FunctionsReminder()


# simulate CMR   
# CMR_sp: serial positions of items recalled during initial recall (if N=1: matches empirical data)
# reminders: serial positions of remaining items (ordered by their wordpool index value)
# recalls: serial positions of items recalled post-cue (only for last repetition of reminder session)
# accs: recall gain for every repetition of reminder session
# pcas: row for each item; columns 0&1 is temporal context, columns 2&3 is semantic context
# pcas2: row for each item; columns 0&1 are combined temporal and semantic context (ie., encoding context)
# net_cs: encoding context vector for every item (ordered by presentation list order in each trial)
CMR_sp,reminders,recalls,accs,pcas,pcas2,net_cs = functions.model_probCMR(N=1,ll=ll,lag_examine=4,data_id=data_id)


# correct for an indexing difference from data and CMR output by finding shown reminder index in CMR's output 
# (as data files are ordered by serial position whereas CMR is order by wordpool index of the shown words
rmdr_idxs_CMR = []; indx = 0
for r in rmdr_idxs_data: 
    r = int(r.item())                        # index in remaining word list in data, ordered by presentation list
    #print(reminders[indx])                  # the serial positions remaining, ordered by wordpool index
    rem_sorted = np.sort(reminders[indx])    # the serial positions remaining, ordered by pres index
    this_reminder = int(rem_sorted[r])       # corresponding sp value
    r = reminders[indx].index(this_reminder) # index of reminder in CMR's sorted list
    rmdr_idxs_CMR.append(r)
    indx+=1
    

# get CMR's average performance gain from cues
post_amts_CMRA = []
for i in range(len(accs)):
    acc = [np.mean(x) for x in accs[i]]
    post_amts_CMRA.append(acc[rmdr_idxs_CMR[i]])

### <p></p>

### Power Analyses

#### 1. before/after ttest

In [4]:
x = recs_amts_data
y = [recs_amts_data[i] + post_amts_data[i] for i in range(ntrials)]

u1 = np.mean(x); n1 = len(x); s1 = np.std(x)
u2 = np.mean(y); n2 = len(y); s2 = np.std(y)
d = (u1 - u2) / (np.sqrt( ((n1-1) * s1**2 + (n2-1) * s2**2 ) / (n1+n2-2)))

power_analysis = TTestIndPower()
sample_size = power_analysis.solve_power(effect_size = d, alpha = 0.05, power = 0.95, alternative = 'smaller')
print(sample_size/12)

28.760247668243426


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: invalid value encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)


#### 2. partial correlation

In [5]:
# log transform
nRemaining = [16-i for i in recs_amts_data]
nRemaining_log = [math.log(x+0.00001) for x in nRemaining]
post_amts_data_log = [math.log(x+0.00001) for x in post_amts_data]
post_amts_CMRA_log = [math.log(x+0.00001) for x in post_amts_CMRA]

x = post_amts_data_log
y = post_amts_CMRA_log
covar = nRemaining_log
data = {'covar':covar, 'x':x, 'y':y}; df = pd.DataFrame(data)
stats = pg.partial_corr(data=df, x='x', y='y', covar='covar', method='spearman').round(3)
r = stats['r'][0]

power_analysis = TTestIndPower()
sample_size = power_analysis.solve_power(effect_size = r, alpha = 0.05, power = 0.95)
print(sample_size/12)

56.457313996339735


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: invalid value encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: invalid value encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)


#### 3. best/worst analysis

In [12]:
from analysis_helpers import perf_by_type
hum_low, hum_rnd, hum_upp, _ = perf_by_type(
    post_amts_data_log, nsubjects, ntrials_persub, exp, list(range(nsubjects)), log=True)

x = hum_low
y = hum_upp

u1 = np.mean(x); n1 = len(x); s1 = np.std(x)
u2 = np.mean(y); n2 = len(y); s2 = np.std(y)
d = (u1 - u2) / (np.sqrt( ((n1-1) * s1**2 + (n2-1) * s2**2 ) / (n1+n2-2)))

power_analysis = TTestIndPower()
sample_size = power_analysis.solve_power(effect_size = d, alpha = 0.05, power = 0.95)
print(sample_size)

193.37097285955292


### <p></p>